In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/NIR/data(markup 1, 2, 3, 5).csv" /content/data.csv

In [3]:
!cp "/content/drive/MyDrive/NIR/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/NIR/eval.csv" /content/eval.csv

In [ ]:
import pandas as pd

train = pd.read_csv("/content/train.csv", sep=';', encoding='utf-8')
train.to_csv('train.csv', index=False, encoding='utf-8')

eval = pd.read_csv("/content/eval.csv", sep=';', encoding='utf-8')
eval.to_csv('eval.csv', index=False, encoding='utf-8')

#### Разделение набора данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("/content/data.csv")
# 500 в eval наборе данных
train, eval = train_test_split(data, test_size=500, random_state=3)

In [ ]:
train.to_csv('train.csv', index=False, encoding='utf-8')
eval.to_csv('eval.csv', index=False, encoding='utf-8')

##### Разметка набора данных train

In [ ]:
from IPython.display import clear_output 
from pprint import pprint

train['result_match'] = 0

# разметка 500 первых новостей train
for row_number, index in enumerate(train.index.tolist()[500:1000]):
  row = train.iloc[index]
  print(row['team1'], ' VS ', row['team2'])
  pprint(row['last_part_content'], width=150)
  print()
  pprint(row['part_body'], width=150)
  label = int(input())
  train.at[index, 'result_match'] = label
  clear_output()

In [ ]:
train['result_match'].value_counts()

0    92408
1       87
Name: result_match, dtype: int64

In [ ]:
train.to_csv('train.csv', index=False, encoding='utf-8')

In [ ]:
eval['result_match'] = 0

# разметка 500 первых новостей eval
for row_number, index in enumerate(eval.index.tolist()[:500]):
  row = eval.iloc[row_number]
  print(row['team1'], ' VS ', row['team2'])
  pprint(row['last_part_content'], width=150)
  print()
  pprint(row['part_body'], width=150)
  label = int(input())
  eval.at[index, 'result_match'] = label
  clear_output()

In [ ]:
eval['result_match'].unique()

array([1, 0])

In [ ]:
eval.to_csv('eval.csv', index=False, encoding='utf-8')

#### Разметка релевантных новостей с активным обучением

In [4]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [5]:
class CustomHuggingFaceDatasets(Dataset):
    """
    Support for `huggingface.datasets`: (https://github.com/huggingface/datasets).
    The purpose of this wrapper is to separate the labels from the rest of the sample information
    and make the dataset ready to be used by `baal.active.ActiveLearningDataset`.
    Args:
        dataset (Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter
            sequences.
    """

    def __init__(
        self,
        dataset,
        tokenizer=None,
        target_key: str = "label",
        input_key_1: str = "sentence1",
        input_key_2: str = "sentence1",
        max_seq_len: int = 512,
    ):
        self.dataset = dataset
        self.targets, self.text1, self.text2 = self.dataset[target_key], self.dataset[input_key_1], self.dataset[input_key_2]
        self.targets_list: List = np.unique(self.targets).tolist()
        self.input_ids, self.attention_masks = (
            self._tokenize(tokenizer, max_seq_len) if tokenizer else ([], [])
        )

    @property
    def num_classes(self):
        return len(self.targets_list)

    def _tokenize(self, tokenizer, max_seq_len):
        # For speed purposes, we should use fast tokenizers here, but that is up to the caller
        tokenized = tokenizer(
            self.text1,
            self.text2,
            add_special_tokens=True,
            max_length=max_seq_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return tokenized["input_ids"], tokenized["attention_mask"]

    def label(self, idx: int, value: int):
        """Label the item.
        Args:
            idx: index to label
            value: Value to label the index.
        """
        self.targets[idx] = value

    def __len__(self):
        return len(self.text1)

    def __getitem__(self, idx):
        target = self.targets_list.index(self.targets[idx])

        return {
            "input_ids": self.input_ids[idx].flatten() if len(self.input_ids) > 0 else None,
            "inputs": self.text1[idx] + '\n' + self.text2[idx],
            "attention_mask": self.attention_masks[idx].flatten()
            if len(self.attention_masks) > 0
            else None,
            "label": torch.tensor(target, dtype=torch.long),
        }

def custom_active_huggingface_dataset(
    dataset,
    tokenizer=None,
    target_key: str = "label",
    input_key_1: str = "sentence1",
    input_key_2: str = "sentence2",
    max_seq_len: int = 512,
    **kwargs
):
    """
    Wrapping huggingface.datasets with baal.active.ActiveLearningDataset.
    Args:
        dataset (torch.utils.data.Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter sequences.
        kwargs (Dict): Parameters forwarded to 'ActiveLearningDataset'.
    Returns:
        an baal.active.ActiveLearningDataset object.
    """

    return ActiveLearningDataset(
        CustomHuggingFaceDatasets(dataset, tokenizer, target_key, input_key_1, input_key_2, max_seq_len), **kwargs
    )

Information on the hyperparms below

* epoch: Number of times you want to run and AL loop
* batch_size: The train and eval batch size for hf trainer arguments
* model: Hugging Face Model
* query_size: Number of samples you want to query at each AL iteration for labelling
* heuristic: The acquisition function/heuristic based on which you want to query the important samples
* iterations: The number of iterations you want to run for MCdropout to find the uncertanities
* shuffle_prop: Additional Noise to counter selection bias
* learning_epoch: Traing epochs for hugging face trainer

In [6]:
hyperparams = {
    "epoch": 4,
    "batch_size": 10,
    "model": "DeepPavlov/rubert-base-cased",
    "query_size": 50,
    "heuristic": "bald",
    "iterations": 10,
    "shuffle_prop": 0.05,
    "learning_epoch": 2,
}

In [7]:
# Check for CUDA
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

id2label = {0: "LABEL_0", 1: "LABEL_1"}
label2id = {"LABEL_0": 0, "LABEL_1": 1}

# Load Model
hf_model = AutoModelForSequenceClassification.from_pretrained(
    hyperparams["model"], num_labels=2, id2label=id2label, label2id=label2id
    )

# Setup tokenizer for model
tokenizer = AutoTokenizer.from_pretrained(hyperparams["model"])

# Enable dropouts for predictions
hf_model = patch_module(hf_model)

# Send model to device and setup cuda arguments
if use_cuda:
    hf_model.to("cuda:0")
    no_cuda = False
else:
    hf_model.to("cpu")
    no_cuda = True

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [8]:
# Define labels in your dataset
label_list = [0, 1]

# Load data from files
data = load_dataset("csv", data_files={'train': '/content/train.csv', 'eval': "/content/eval.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2dd5fa8b82495966/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def get_label_from_data(active_dataset, data, target, input1, input2, indexes) -> List[int]:
    """
    Get labels from the active dataset, this assumes that you have
    already labelled some samples in your initial dataset

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """

    labels = []

    # Now since you labelled points earlier now some part of pool has become train
    # so in order to get the pool indexes based on your 'original' data i.e
    # your raw_train_set. Make sure to user __pool_tp

    raw_data_idx = active_dataset._pool_to_oracle_index(indexes)

    for idx in raw_data_idx:
        #print(f"Adding labels for Raw data Index {idx} : {data[input1][idx]}")
        #print(f"Adding labels for Raw data Index {idx} : {data[input2][idx]}")

        label = data[target][idx]
        labels.append(label)
        #print(label)
        #print("\n")

    return labels

In [10]:
def get_label_human_oracle(active_dataset, indexes) -> List[int]:
    """
    Get labels from human oracle. During the AL loop some samples
    will go to the human labeller

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """
    # List for corresponding labels
    labels = []

    skipped = []
    
    for sample_idx, idx in enumerate(indexes):

        while True:
            try:
                print(idx)
                pprint(active_dataset.pool.__getitem__(idx)['inputs'], width=150)
                label = int(input())
            except ValueError:
                print("Sorry, I didn't understand that.")
                continue
            if label != -1 and label not in label_list:
                print(f"Allowed labels are {label_list}")
                continue
            if label == -1:
                print("Skipping this sample")
                skipped.append(sample_idx)
                break
            else:
                labels.append(label)
                break
        print("\n")

    indexes_upd = np.delete(indexes, skipped)

    return labels, indexes_upd

In [11]:
#small_data = data['train'].select(range(10_000))
small_data = data['train']

active_set = custom_active_huggingface_dataset(
    small_data,
    tokenizer=tokenizer,
    target_key='result_match',
    input_key_1='last_part_content',
    input_key_2="part_body"
)

active_set.can_label = True

# 1000 первых объектов набора данных
label_from_data = get_label_from_data(active_set, small_data, 'result_match', 'last_part_content', 'part_body', range(500))

active_set.label(
    range(500),
    label_from_data,
)

valid_set = CustomHuggingFaceDatasets(data['eval'], tokenizer=tokenizer, target_key='result_match', input_key_1='last_part_content', input_key_2="part_body")

active_set, test_set = active_set, valid_set

In [12]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

# Model save checkpoint
save_checkpoint = 2

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [13]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")

    if (epoch != last_epoch):
      # MCdropout to gather uncertanities
      predictions = baal_trainer.predict_on_dataset(
          active_set.pool, iterations=hyperparams["iterations"]
      )
      print("\n")

      # Acquistion of the most informative and diverse samples based on BatchBALD
      top_uncertainty = heuristic(predictions)[: hyperparams.get("query_size", 1)]

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, top_uncertainty
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.601100,0.549956,0.745098
2,0.363100,0.616603,0.760369




[158-MainThread  ] [baal.transformers_trainer_wrapper:predict_on_dataset_generator:67] 2023-04-14T04:03:41.575951Z [info     ] Start Predict                  dataset=14090



100%|██████████| 1409/1409 [1:18:20<00:00,  3.34s/it]




1858
('"Рубин" стоя катает мяч. В общем, бежать вперед сейчас смысла действительно нет, а "Спартак" как-то не рвется вступать в отбор. Касаев забежал в '
 'офсайд, нанес удар после свистка и получил желтую. Дзюба появляется вместо де Зеува. Карадениз красавец, двух защитников накрутил на фланге и '
 'простреливал вдоль ворот. Угловой. Рязанцев, проведя очень хороший матч, садится отдыхать. Лебеденко вместо него. Режиссер удачно поймал в кадр '
 'грустные глаза Кариоки. Очень точно они отражают расклад по сегодняшней игре: на "Спартак" смотреть было грустно. Ну а "Рубин" уже начал '
 'переигрывать, футболисты стараются провести и завершить атаку как можно изящнее. Карадениз - тоже, кстати, большой молодец сегодня - заменен на '
 'Алексея Еременко. "Рубин" даже в расслабленном состоянии не позволяет сопернику провести внятной атаки. Веллитон неплохо выкатил в штрафную на '
 'Кирилла Комбарова, а того опередил защитник. Кирилл обреченно потрусил обратно к своей штрафной. Алексей Еременк

...


100%|██████████| 1399/1399 [1:17:51<00:00,  3.34s/it]




3288
('Ароника вспомнил уроки Ибрагимовича и отмахнулся от Дрогба. Дрогба, конечно, тоже хорош =) Замена: вместо Ароники выходит Варгас. Инлер получает '
 '"горчичник" за фол на Торресе. Вновь "Челси" зарабатывает штрафной на половине поля соперника. Время идет, шансы "Наполи" тают. Дрогба поймал '
 'кураж. Теперь от него отмахнулся Кампаньяро! Нет, ну это, право, талант! Желтая карточка игроку "Наполи". Этому Дидье еще в "Марселе" научился =) '
 'Дрогба со штрафного угодил в стенку... "Наполи" пытается организовать штурм, но пока неудачно. Давид Луиз сел на газон. Арбитр не верит в его '
 'страдания, и игра продолжается. Замен-то у "Челси" нет. "Челси" переключился на игру "на отбой". Пошла 120-я минута. Два в одного вышли Малуда и '
 'Дрогба, Дидье бил с лета - мимо ворот! Плюс две минуты к 120. Все! 4:1! "Челси" совершил нет, не чудо, но что-то в этом духе! Поздравления '
 'болельщикам "Челси"! Блестящая победа! Ди Маттео не только вывел команду в четвертьфинал - "Челси" сыграл, к

 75%|███████▌  | 3/4 [4:21:23<1:27:17, 5237.19s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.548400,0.447504,0.816733
2,0.385500,0.386357,0.841085


100%|██████████| 4/4 [4:24:26<00:00, 3966.72s/it]

In [14]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-130")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.35900914669036865,
 'eval_f1': 0.8512241054613936,
 'eval_runtime': 15.9423,
 'eval_samples_per_second': 31.363,
 'eval_steps_per_second': 3.136}

In [15]:
!cp -r "/content/checkpoint-130" "/content/drive/MyDrive/NIR/bald"

#### Разметка релевантных новостей со случайным обучением

In [ ]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [ ]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")
    if (epoch != last_epoch):
      point_idx_train = np.random.choice(len(active_set.pool) - 1, hyperparams.get("query_size", 1), replace=False)

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, point_idx_train
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # Save model
      #if epoch == save_checkpoint:
      #    save_model(baal_trainer)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,F1
1,0.712300,0.707606,0.667557
2,0.635200,0.527398,0.767591




7803
('И в НХЛ бывают проблемы с техникой. С часами нелады на арене. Импровизированный длинный тайм-аут получился в итоге, после которого хозяева сняли '
 'кипера. Пару раз гости перехватывали шайбу, но скорее выезжали с ней в среднюю зону, чтобы не придумывать пробросы и вбрасывания. Все, Ови '
 'помогает своей команде одержать важную в турнирном плане победу, забросив две шайбы в большинстве! Броски - 38:31, еще больше увеличили свое '
 'преимущество хозяева в 3 периоде. Вбрасывания так вообще - 38:25, один Бержерон чего стоит 14:7, хотя и проиграл пару важных в конце. "Вашингтону" '
 'очень нужна была эта победа. Вот такая. Когда сыграй прямо сейчас еще 10 матчей, вряд ли "Кэпиталс" выиграли бы больше 2. Но вовремя забрасывал '
 'Ови, вовремя отстояли почти 2 минуты 3 против 5. Вовремя перехватывали шайбы Уорд и Фер. И в обороне сыграли классно, не дали "Бостону" '
 'развернуться. Если по цифрам, то 24 смены у Ови и 19:52 на льду, 1 хит. 9 бросков, 2 гола и -1. У Орлова 2 броска, 

 25%|██▌       | 1/4 [10:39<31:58, 639.40s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.729200,0.693354,0.604200
2,0.708900,0.703574,0.550523


...

Epoch,Training Loss,Validation Loss,F1
1,0.703300,0.706445,0.007843
2,0.715600,0.691195,0.623729




10300
('В штрафной "Герты" становится жарко! Швайнштайгер тоже хорош в верховой игре. Бастиан попал в штангу.   ГОООЛ! 1:0! Баастиан Швааайнштайгер! '
 '«Герта» как-то совсем уж зажалась в обороне, и это ничем хорошим не закончилось. Вайзер прошел по правому флангу, откатил мяч назад, а '
 'Швайнштайгер расстрелял ворота из центра штрафной. Вагнер вышел вместо Шульца. Дардаи снял с игры трех футболистов, которые сегодня были наиболее '
 'активны. Надо отметить, те замены, которые венгр делал при счете 0:0, никак не помогли "Герте" разжиться набранными очками. Вагнер умело '
 'развернулся с мячом в чужой штрафной, но акцентированного удара или опасного прострела не получилось, Нойер легко справился. Швайнштайгер мог '
 'оформить дубль! Бастиан бил правой ногой из пределов штрафной в дальний нижний угол. Бурхерт вытащил.  "Герта" организовала навал с обилием '
 'верховых передач. Борьба в штрафной "Баварии" велась. Опасных ударов и близко не было. Матч окончен. "Бавария" кладет в копил

 75%|███████▌  | 3/4 [28:11<09:10, 550.67s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.725200,0.595924,0.616580
2,0.663000,0.496432,0.791252


100%|██████████| 4/4 [31:17<00:00, 469.32s/it]

In [ ]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-130")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.4987184703350067,
 'eval_f1': 0.7917448405253283,
 'eval_runtime': 15.9751,
 'eval_samples_per_second': 31.299,
 'eval_steps_per_second': 3.13}

In [ ]:
!cp -r "/content/checkpoint-130" "/content/drive/MyDrive/NIR/random"